In [25]:
#### Import packages
import sqlite3
import os
import requests

In [ ]:

## Be sure you can print the os package environment is set to the correct location, where the API key is stored
print(os.environ['HOME'])
### Set API Key
map_pass = os.environ.get('MAP_PASS') 


In [26]:
### Connect to the database
connection = sqlite3.connect("database.sqlite")

In [27]:
# cursor  
crsr = connection.cursor() 

In [28]:
from sqlite3 import Error

## Check connection
def sql_connection():
    try:
        con = sqlite3.connect(':memory:')
        print("Connection is established: Database is created in memory")
    except Error:
        print(Error)
    finally:
        con.close()
sql_connection()


Connection is established: Database is created in memory


In [34]:
## Print all table names to preview table content
def sql_fetch(con):
    cursorObj = con.cursor() 
    cursorObj.execute('SELECT name from sqlite_master where type= "table"')
    tables = cursorObj.fetchall()
    return tables

tables = sql_fetch(connection)
print(tables)

[('sqlite_sequence',), ('Player_Attributes',), ('Player',), ('Match',), ('League',), ('Country',), ('Team',), ('Team_Attributes',), ('geocodes',)]


In [49]:
### create the geocode table-- 
## with foreign keys for country name and country id 
## put in a check on if the table exists, so it won't throw an error

def sql_table(con): 
    cursorObj = con.cursor() 
    tables = sql_fetch(connection)
    if ('geocodes',) in tables:
        print("Table already exixts")
    else:
        cursorObj.execute("CREATE TABLE geocodes(id integer PRIMARY KEY, country_id integer, country_name text, lat real, long real, CONSTRAINT fk_country_id FOREIGN KEY (country_id) REFERENCES Country(id),CONSTRAINT fk_country_name FOREIGN KEY (country_name) REFERENCES Country(name))")

    con.commit()

sql_table(connection)

## check the tables to be sure it has been added
sql_fetch(connection)

Table already exixts


[('sqlite_sequence',),
 ('Player_Attributes',),
 ('Player',),
 ('Match',),
 ('League',),
 ('Country',),
 ('Team',),
 ('Team_Attributes',),
 ('geocodes',)]

In [76]:
### Get values for the country table, to see can be sure they link to the new geocode table
def sql_fetch_countries(con):
    cursorObj = con.cursor() 
    cursorObj.execute('SELECT * FROM Country')
    countries= cursorObj.fetchall()
    return countries

countries = sql_fetch_countries(connection)
print(countries)

[(1, 'Belgium'), (1729, 'England'), (4769, 'France'), (7809, 'Germany'), (10257, 'Italy'), (13274, 'Netherlands'), (15722, 'Poland'), (17642, 'Portugal'), (19694, 'Scotland'), (21518, 'Spain'), (24558, 'Switzerland')]


In [107]:
## test out a request, to get lat and long of the first result of the search word, limited to countries and using the local environment variable
country = "England"

response = requests.get("https://api.mapbox.com/geocoding/v5/mapbox.places/{}.json?access_token={}".format(country,map_pass))
data = response.json()

### Save the latitude and longitude variables to put in the geocodes table
long = data['features'][0]['center'][0]
lat = data['features'][0]['center'][1]

In [109]:
## Check that the values are correct
print("longitude:", data['features'][0]['center'][0], long)
print("latitude:", data['features'][0]['center'][1], lat)

longitude: -0.11667 -0.11667
latitude: 51.5 51.5


In [115]:
## create search list
ids = []
words = []
countries = sql_fetch_countries(connection)
for a,b in countries:
    words.append(b)
    ids.append(a)

print(words)
print(ids)

['Belgium', 'England', 'France', 'Germany', 'Italy', 'Netherlands', 'Poland', 'Portugal', 'Scotland', 'Spain', 'Switzerland']
[1, 1729, 4769, 7809, 10257, 13274, 15722, 17642, 19694, 21518, 24558]


In [116]:
### for each country name, perform a get request and store the lat long variables
long = []
lat = []
for i in words:
    response = requests.get("https://api.mapbox.com/geocoding/v5/mapbox.places/{}.json?access_token={}".format(i,map_pass))
    data = response.json()
    long.append(data['features'][0]['center'][0])
    lat.append(data['features'][0]['center'][1])

data_tuples = list(zip(ids,words,lat,long))

print(data_tuples)


[(1, 'Belgium', 51, 5), (1729, 'England', 51.5, -0.11667), (4769, 'France', 47, 2), (7809, 'Germany', 51, 10), (10257, 'Italy', 43, 12), (13274, 'Netherlands', 52.31667, 5.55), (15722, 'Poland', 52, 19), (17642, 'Portugal', 38.7, -9.18333), (19694, 'Scotland', 57, -5), (21518, 'Spain', 40, -3), (24558, 'Switzerland', 46.79856, 8.23197)]


In [118]:
def add_table_values(con, data_tuples):
    cursorObj = con.cursor() 
    i =1
    for a,b,c,d in data_tuples:
        print(i,a,b,c,d)
        cursorObj.execute('INSERT INTO geocodes (id,country_id, country_name, lat, long) VALUES (?,?,?,?,?)', (i,a,b,c,d))
        i += 1
    con.commit()

add_table_values(connection, data_tuples)

1 Belgium 51 5
1729 England 51.5 -0.11667
4769 France 47 2
7809 Germany 51 10
10257 Italy 43 12
13274 Netherlands 52.31667 5.55
15722 Poland 52 19
17642 Portugal 38.7 -9.18333
19694 Scotland 57 -5
21518 Spain 40 -3
24558 Switzerland 46.79856 8.23197


In [119]:
### Get values for the geocodes table, to see how to link to the new geocode table
def sql_fetch_rows(con):
    cursorObj = con.cursor() 
    cursorObj.execute('SELECT * FROM geocodes')
    geos= cursorObj.fetchall()
    return geos

sql_fetch_rows(connection)


[(1, 1, 'Belgium', None, None),
 (2, 1729, 'England', None, None),
 (3, 4769, 'France', None, None),
 (4, 7809, 'Germany', None, None),
 (5, 10257, 'Italy', None, None),
 (6, 13274, 'Netherlands', None, None),
 (7, 15722, 'Poland', None, None),
 (8, 17642, 'Portugal', None, None),
 (9, 19694, 'Scotland', None, None),
 (10, 21518, 'Spain', None, None),
 (11, 24558, 'Switzerland', None, None),
 (12, 1, 'Belgium', 51.0, 5.0),
 (13, 1729, 'England', 51.5, -0.11667),
 (14, 4769, 'France', 47.0, 2.0),
 (15, 7809, 'Germany', 51.0, 10.0),
 (16, 10257, 'Italy', 43.0, 12.0),
 (17, 13274, 'Netherlands', 52.31667, 5.55),
 (18, 15722, 'Poland', 52.0, 19.0),
 (19, 17642, 'Portugal', 38.7, -9.18333),
 (20, 19694, 'Scotland', 57.0, -5.0),
 (21, 21518, 'Spain', 40.0, -3.0),
 (22, 24558, 'Switzerland', 46.79856, 8.23197)]